In [1]:
import json
import dotenv
import os
import requests
from constants import *
from graph_constructor import construct_pool_graph, pool_graph_to_dict
from pathfinder import find_shortest_paths, validate_all_paths, create_path_graph, path_graph_to_dict
from path_crawler import calculate_routes, get_final_route

# open pool_dict from data
with open('data/pool_dict.json') as f:
    pool_dict = json.load(f)

DEX_LIST = (
    UNISWAP_V2,
    UNISWAP_V3,
    SUSHISWAP_V2,
    CURVE,
    BALANCER_V1,
    BALANCER_V2,
    DODO,
    PANCAKESWAP_V3
)

DEX_LIQUIDITY_METRIC_MAP = {
    UNISWAP_V2: 'reserveUSD',
    UNISWAP_V3: 'totalValueLockedUSD',
    SUSHISWAP_V2: 'liquidityUSD',
    CURVE: 'reserveUSD',
    BALANCER_V1: 'reserveUSD',
    BALANCER_V2: 'reserveUSD',
    DODO: 'reserveUSD',
    PANCAKESWAP_V3: 'totalValueLockedUSD'
}

# load coinmarketcap api key from .env
dotenv.load_dotenv()
COINMARKETCAP_API_KEY = os.getenv("COINMARKETCAP_API_KEY")

def get_token_price_usd(symbol, convert_to_symbol="USD"):
    url = f"https://pro-api.coinmarketcap.com/v1/tools/price-conversion?amount=1&symbol={symbol}&convert={convert_to_symbol}"
    headers = {
        "Accepts": "application/json",
        "X-CMC_PRO_API_KEY": COINMARKETCAP_API_KEY
    }
    response = requests.get(url, headers=headers)
    response_data = json.loads(response.text)
    return response_data["data"]["quote"][convert_to_symbol]["price"]

def find_max_liquidity(pool_dict, n=10):
    most_liquid_pools = sorted(pool_dict.items(), key=lambda x: float(x[1][DEX_LIQUIDITY_METRIC_MAP[x[1]['protocol']]]), reverse=True)[:n]
    return float(most_liquid_pools[0][1][DEX_LIQUIDITY_METRIC_MAP[most_liquid_pools[0][1]['protocol']]])

def find_max_price(pool_dict, n=10):
    most_expensive_pools = sorted(pool_dict.items(), key=lambda x: max(float(x[1]['token0'].get('priceUSD', 0)), float(x[1]['token1'].get('priceUSD', 0))), reverse=True)[:n]
    return max(float(most_expensive_pools[0][1]['token0'].get('priceUSD', 0)), float(most_expensive_pools[0][1]['token1'].get('priceUSD', 0)))

def calculate_score(pool, trade_value, max_trade_value, max_price, max_liquidity):
    # Normalize the factors
    normalized_price = max(float(pool['token0'].get('priceUSD', 0)), float(pool['token1'].get('priceUSD', 0))) / max_price
    normalized_liquidity = float(pool[DEX_LIQUIDITY_METRIC_MAP[pool['protocol']]]) / max_liquidity

    # Adjust the weights based on the trade value
    w1 = trade_value / max_trade_value  # Liquidity weight increases with trade value
    w2 = 1 - w1  # Price weight decreases with trade value

    # Calculate the score
    score = w1 * normalized_liquidity + w2 * normalized_price

    return score
#              filter_pools(sell_symbol: str, sell_ID: str, buy_symbol: str, buy_ID: str, exchanges=None, X: int = 50) -> list:
def filter_pools_best_match(sell_symbol: str, sell_ID: str, sell_amount: float, exchanges=None, X: int = 25, Y: int = 25):
    global pool_dict
    # Calculate trade value
    avg_sell_token_price_usd = get_token_price_usd(sell_symbol)
    trade_value = sell_amount * avg_sell_token_price_usd
    max_trade_value = 100000000  # Arbitrarily set
    
    # Get max_price and max_liquidity from all pools
    max_price = find_max_price(pool_dict)
    max_liquidity = find_max_liquidity(pool_dict)
    
    pool_scores = []
    for pool in pool_dict.values():
        # Exclude pools whose exchange is not in the exchanges list
        if exchanges is not None and pool['protocol'] not in exchanges:
            continue
        
        # Calculate pool score
        score = calculate_score(pool, trade_value, max_trade_value, max_price, max_liquidity)
        pool_scores.append((pool, score))

    # Sort the pools based on score
    sorted_pool_scores = sorted(pool_scores, key=lambda x: x[1], reverse=True)
    # Get the top X pools by score
    top_X_pools = [pool_score[0] for pool_score in sorted_pool_scores[:X]]
    # Get the top Y pools that contain the sell_token
    top_Y_sell_token_pools = [pool_score[0] for pool_score in sorted_pool_scores if sell_ID in (pool_score[0]['token0']['id'], pool_score[0]['token1']['id'])][:Y]
    # Return a list combining the two sets of pools
    return top_X_pools + top_Y_sell_token_pools

def route_orders(sell_symbol: str, sell_ID: str, sell_amount: float, buy_symbol: str, buy_ID: str, exchanges, split=False, routing_strategy='default') -> dict:
    result = {}
    
    # get the pools
    if routing_strategy == 'best_match':
        filt_pools = filter_pools_best_match(sell_symbol, sell_ID, sell_amount, exchanges=exchanges)
    else:
        filt_pools = filter_pools(sell_symbol, sell_ID, buy_symbol, buy_ID, exchanges=exchanges)
    
    if len(filt_pools) < 5:
        time.sleep(5)
        filter_pools(sell_symbol, sell_ID, buy_symbol, buy_ID, exchanges=DEX_LIST)
    
    # construct the pool graph
    G = construct_pool_graph(filt_pools)
    
    # get the graph dict
    graph_dict = pool_graph_to_dict(G)
    
    # append the dict to the result
    result['pool_graph'] = graph_dict
    
    # find the shortest paths
    paths = find_shortest_paths(G, sell_symbol, buy_symbol)
    
    # validate the paths
    if routing_strategy == 'best_match': # all paths are valid if it doesn't matter what our output token is, any paths that somehow still involve selling a token to a pool not accepted will be filtered out by the price impact calculation anyway
        valid_paths = paths
    else:
        valid_paths = validate_all_paths(G, paths, sell_ID, buy_ID)
    
    # create the path graph
    path_graph = create_path_graph(valid_paths)
    
    # get the path graph dict
    path_graph_dict = path_graph_to_dict(path_graph)
    
    # append the dict to the result
    result['path_graph'] = path_graph_dict
    
    # calculate the routes (traverse the paths and calculate price impact at each swap)
    routes = calculate_routes(G, valid_paths, sell_amount, sell_symbol, buy_symbol)
    
    if split:
        final_route = get_final_route(G, routes, sell_amount, sell_symbol)
        result['routes'] = final_route
    else:
        result['routes'] = routes[:MAX_ROUTES]

    return result

In [2]:
# test best match without splitting
unsplit = route_orders("MKR", "0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2", 100, "USDC", "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", ('Uniswap_V2', 'Uniswap_V3', 'Sushiswap_V2', 'Curve', 'DODO', 'PancakeSwap_V3'), split=False, routing_strategy='best_match')

float division by zero
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell 

In [3]:
unsplit

{'pool_graph': {'GP_WETH_0x7e3a3a525d9d265d11d1d1db3cad678746b47d09': ['GP_WETH_0x7e3a3a525d9d265d11d1d1db3cad678746b47d09',
   'TFT_WETH_0x43c4ea688e53aef3aab172b537b5df6d82f4c4dc',
   'PUNKS_WETH_0xa80ccc104349d2ee29998c54d6e6488012f8afe0',
   'DSD_WETH_0x7206ae46531093cf2dbbab4c74ac6135e6535a91',
   'WETH_CRT_0xe8202cccda10a211cc9a5d6a0157fb47ba2ab157',
   'KCC_WETH_0xec3f403b6b23462d8d657a6a24aef59d3cd622b9',
   'tiger_WETH_0x207a0856d0cc2648260d05dc5cb3ba78bd5022f9',
   'EGOD_WETH_0xfb57be17b95872fcea940ddfe98dcecdf2ac8377',
   'JZo2o_WETH_0xcee2e90275af0aa1ac425dd8a41e3857f7acc6ba',
   'USDC_WETH_0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
   'USDC_WETH_0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
   'WBTC_WETH_0xcbcdf9626bc03e24f779434178a73a0b4bad62ed',
   'MKR_WETH_0xe8c6c9227491c0a8156a0106a0204d881bb7e531',
   'MKR_WETH_0x3afdc5e6dfc0b0a507a8e023c9dce2cafc310316',
   'MKR_WETH_0xc2adda861f89bbb333c90c492cb837741916a225',
   'MKR_WETH_0xba13afecda9beb75de5c56bbaf696b880a5a50

In [4]:
split = route_orders("MKR", "0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2", 100, "USDC", "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", ('Uniswap_V2', 'Uniswap_V3', 'Sushiswap_V2', 'Curve', 'DODO', 'PancakeSwap_V3'), split=True, routing_strategy='best_match')

float division by zero
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell token not accepted by pool: 0x936198fcac6d8cdec3815f24ba250041f593f6c3
string indices must be integers
Sell token not accepted by pool: 0xc486ad2764d55c7dc033487d634195d6e4a6917e
string indices must be integers
Sell 

In [5]:
split

{'pool_graph': {'GP_WETH_0x7e3a3a525d9d265d11d1d1db3cad678746b47d09': ['GP_WETH_0x7e3a3a525d9d265d11d1d1db3cad678746b47d09',
   'TFT_WETH_0x43c4ea688e53aef3aab172b537b5df6d82f4c4dc',
   'PUNKS_WETH_0xa80ccc104349d2ee29998c54d6e6488012f8afe0',
   'DSD_WETH_0x7206ae46531093cf2dbbab4c74ac6135e6535a91',
   'WETH_CRT_0xe8202cccda10a211cc9a5d6a0157fb47ba2ab157',
   'KCC_WETH_0xec3f403b6b23462d8d657a6a24aef59d3cd622b9',
   'tiger_WETH_0x207a0856d0cc2648260d05dc5cb3ba78bd5022f9',
   'EGOD_WETH_0xfb57be17b95872fcea940ddfe98dcecdf2ac8377',
   'JZo2o_WETH_0xcee2e90275af0aa1ac425dd8a41e3857f7acc6ba',
   'USDC_WETH_0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
   'USDC_WETH_0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
   'WBTC_WETH_0xcbcdf9626bc03e24f779434178a73a0b4bad62ed',
   'MKR_WETH_0xe8c6c9227491c0a8156a0106a0204d881bb7e531',
   'MKR_WETH_0x3afdc5e6dfc0b0a507a8e023c9dce2cafc310316',
   'MKR_WETH_0xc2adda861f89bbb333c90c492cb837741916a225',
   'MKR_WETH_0xba13afecda9beb75de5c56bbaf696b880a5a50